In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
import os


#import hiddenlayer as hl

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [3]:
batch_size = 16
learning_rate = 0.9

transforms = transforms.Compose(
[transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Training', transform=transforms)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Test', transform=transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#def imshow(inp, title=None):
    
  #  inp = inp.cpu() if device else inp
  #  inp = inp.numpy().transpose((1, 2, 0))
    
 #   mean = np.array([0.485, 0.456, 0.406])
 #   std = np.array([0.229, 0.224, 0.225])
 #   inp = std * inp + mean
 #   inp = np.clip(inp, 0, 1)
    
  #  plt.imshow(inp)
  #  if title is not None:
  ##   plt.pause(0.001)
    
#images, labels = next(iter(train_dataloader)) 
#print("images-size:", images.shape)

#out = torchvision.utils.make_grid(images)
#print("out-size:", out.shape)

#imshow(out, title=[train_dataset.classes[x] for x in labels])

In [4]:
class VGG16_model(nn.Module):
    def __init__(self):
        super(VGG16_model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU()
            )
        
        
        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU()
            )
        
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU()
            )
        
        self.fc3 = nn.Sequential(
            nn.Linear(4096, 131)
            
            )
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        out = self.maxpool(out)

        out = self.layer3(out)
        out = self.layer4(out)

        out = self.maxpool(out)

        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)

        out = self.maxpool(out)

        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)

        out = self.maxpool(out)

        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)

        out = self.maxpool(out)

        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = F.dropout(out, 0.5) #reduce overfitting
        out = self.fc2(out)
        out = F.dropout(out, 0.5)
        out = self.fc3(out)

        return out
        

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(device)
model = VGG16_model() 
model = model.to(device=device)

## Loss and optimizer
learning_rate=0.01
load_model = True
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)#, momentum=0.9) 

#def accuracy(out, labels):
  #  _,pred = torch.max(out, dim=1)
 #   return torch.sum(pred==labels).item()

cuda


In [ ]:
for epoch in range(1): 
    running_loss = 0.0
    correct = 0
    total=0
    for batch_idx, (data_, target_) in enumerate(train_dataloader):
        data_, target_ = data_.to(device), target_.to(device)
        
        ## Forward Pass
        
        outp = model(data_)
        loss = criterion(outp,target_)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _,pred = torch.max(outp, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
    print(f"Loss in epoch {epoch} : {running_loss/len(train_dataloader)}")

   

In [ ]:
torch.save(model.state_dict(), 'vgg.pt')

In [ ]:
def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()